**Importing required libraries**

In [1]:
import pandas as pd
import numpy as np
import folium
import geocoder
from geopy.geocoders import Nominatim
import json
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm 
import matplotlib.colors as colors 
from pandas.io.json import json_normalize
import warnings

warnings.filterwarnings('ignore')
print('done')

done


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).text

**Using BeautifulSoup to convert the request results into xml**

In [4]:
soap = BeautifulSoup(results,'lxml')

In [5]:
data = soap.find('table',{'class':'wikitable sortable'}).findAll('td')
print('total number of postal code records including Not Assigned : ', len(data)/3)

total number of postal code records including Not Assigned :  289.0


In [6]:
postal_code = []
borough = []
neighborhood = []
i=0
while i < len(data):
    if data[i+1].contents[0] !='Not assigned':
        if data[i].contents[0] in postal_code:
            index = postal_code.index(data[i].contents[0])
            neighborhood[index] = neighborhood[index] + ', ' + data[i+2].find('a').get('title') if data[i+2].find('a') else data[i+1].find('a').get('title')
        else:
            postal_code.append(data[i].contents[0])
            temp_borough = data[i+1].find('a').get('title') if data[i+1].find('a') else data[i+1].contents[0]
            borough.append(temp_borough)
            hood = data[i+2].find('a').get('title') if data[i+2].find('a') else data[i+2].contents[0]
            neighborhood.append(hood if hood !='Not assigned' else temp_borough)
    i = i+3
print('Extracting values from data is done')        

Extracting values from data is done


In [7]:
postal_data = pd.DataFrame()
postal_data['PostalCode'] = postal_code
postal_data['Borough'] = borough
postal_data['Neighborhood'] = neighborhood
postal_data.shape

(103, 3)

In [8]:
postal_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Not assigned


In [9]:
postal_data[postal_data['PostalCode'] == 'M5V']['Neighborhood']

87    Downtown Toronto, King and Spadina, Railway La...
Name: Neighborhood, dtype: object

In [10]:
lon_lat = pd.read_csv('Geospatial_Coordinates.csv')
print('total unique postal values :', lon_lat['Postal Code'].nunique())
lon_lat.head()

total unique postal values : 103


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
latitude = []
longitude = []
postal = postal_data['PostalCode'].values
for value in postal:
    temp = lon_lat[lon_lat['Postal Code'] == value]
    latitude.append(temp['Latitude'].values[0])
    longitude.append(temp['Longitude'].values[0])
    
postal_data['Latitude'] = latitude
postal_data['Longitude'] = longitude

postal_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,Downtown Toronto,43.657162,-79.378937
